In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import csv

In [3]:
columnsToUse=["CDIVMSAR", "CENSUS_D", "CENSUS_R", "DRVRCNT", "EDUC", "HHRESP", "HHSIZE", "HHSTATE", "HHSTFIPS", "HHVEHCNT", "HH_CBSA", "HH_HISP", "HH_RACE", "HOMEOWN", "HOUSEID", "HHFAMINC", "MSACAT", "MSASIZE", "NUMADLT", "PRMACT", "PROXY", "RAIL", "R_AGE", "R_AGE_IMP", "R_SEX", "R_SEX_IMP", "SMPLSRCE", "URBAN", "URBANSIZE", "URBRUR", "WRKCOUNT","TRIPPURP"]
df = pd.read_csv('C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\NHTS Data Parser\\HH Data Parser\\trippub.csv')
X = df.drop(columns=['TRIPPURP'])
X=df.drop(columns=columnsToUse)
y = df['TRIPPURP']

In [4]:
non_int64_columns = df.dtypes[df.dtypes != 'int64']
print(non_int64_columns)

TRPMILES    float64
TRIPPURP     object
VMT_MILE    float64
HHSTATE      object
GASPRICE    float64
HH_CBSA      object
WTTRDFIN    float64
TRPMILAD    float64
OBHUR        object
DBHUR        object
dtype: object


In [8]:
label_encoder = LabelEncoder()
# X['TRIPPURP'] = label_encoder.fit_transform(X['TRIPPURP'])
# X['HHSTATE'] = label_encoder.fit_transform(X['HHSTATE'])
# X['HH_CBSA'] = label_encoder.fit_transform(X['HH_CBSA'])
X['OBHUR'] = label_encoder.fit_transform(X['OBHUR'])
X['DBHUR'] = label_encoder.fit_transform(X['DBHUR'])

In [9]:
# Preprocess the target variable using LabelEncoder
y_label_encoder = LabelEncoder()
y_encoded = y_label_encoder.fit_transform(y)

# Convert the target variable to one-hot encoded format
num_classes = len(y_label_encoder.classes_)
y_one_hot = pd.get_dummies(y_encoded)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)
# X_train=X.astype('float64')
num_classes
# Standardize the features (optional, but often recommended for neural networks)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
input_shape = X_train.shape[1]

In [11]:
print(X_train_scaled.shape)
print(y_train.shape)

(738857, 83)
(738857, 6)


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Create the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# model = Sequential()

# # Add layers to the model
# model.add(Dense(64, activation='relu', input_shape=(input_shape,)))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=3, batch_size=32, validation_data=(X_test_scaled, y_test))
# model.fit(X_train, y, epochs=3, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/3
23090/23090 [==============================] - 17s 712us/step - loss: 0.2509 - accuracy: 0.9111 - val_loss: 0.0665 - val_accuracy: 0.9787
Epoch 2/3
23090/23090 [==============================] - 16s 705us/step - loss: 0.0570 - accuracy: 0.9836 - val_loss: 0.0362 - val_accuracy: 0.9931
Epoch 3/3
5773/5773 [==============================] - 3s 509us/step - loss: 0.0235 - accuracy: 0.9960
Test loss: 0.023497819900512695
Test accuracy: 0.9959830045700073


In [13]:
non_int64_columns = df.dtypes[df.dtypes != 'int64']
print(non_int64_columns)

TRPMILES    float64
TRIPPURP     object
VMT_MILE    float64
HHSTATE      object
GASPRICE    float64
HH_CBSA      object
WTTRDFIN    float64
TRPMILAD    float64
OBHUR        object
DBHUR        object
dtype: object


In [14]:
results = []

for feature in X.columns:
    X_modified = X.drop(columns=[feature])
    

    X_train, X_test, y_train, y_test = train_test_split(X_modified, y_one_hot, test_size=0.2, random_state=42)
    scaler = MinMaxScaler()
    X_modified_train_scaled = scaler.fit_transform(X_train)
    X_modified_test_scaled = scaler.transform(X_test)
    # print(X_train.shape)
    # print(y_train.shape)
    model = Sequential()
    model.add(Dense(64, input_dim=X_modified_train_scaled.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_modified_train_scaled, y_train, epochs=3, batch_size=32, validation_data=(X_modified_test_scaled, y_test))

    # model.fit(X_train, y_train)
    # accuracy_dropped = model.score(X_test, y_test)
    accuracy = 0.9959830045700073
    loss, accuracy_dropped = model.evaluate(X_modified_test_scaled, y_test)
    results.append((feature, accuracy - accuracy_dropped))

Epoch 1/3
23090/23090 [==============================] - 17s 729us/step - loss: 0.2841 - accuracy: 0.8969 - val_loss: 0.0614 - val_accuracy: 0.9863
Epoch 2/3
23090/23090 [==============================] - 17s 729us/step - loss: 0.0516 - accuracy: 0.9845 - val_loss: 0.0403 - val_accuracy: 0.9861
Epoch 3/3
5773/5773 [==============================] - 3s 500us/step - loss: 0.0202 - accuracy: 0.9959
Epoch 1/3
23090/23090 [==============================] - 17s 714us/step - loss: 0.2344 - accuracy: 0.9185 - val_loss: 0.0492 - val_accuracy: 0.9884
Epoch 2/3
23090/23090 [==============================] - 17s 728us/step - loss: 0.0492 - accuracy: 0.9856 - val_loss: 0.0244 - val_accuracy: 0.9957
Epoch 3/3
5773/5773 [==============================] - 4s 751us/step - loss: 0.0169 - accuracy: 0.9966
Epoch 1/3
23090/23090 [==============================] - 19s 819us/step - loss: 0.2569 - accuracy: 0.9096 - val_loss: 0.0496 - val_accuracy: 0.9862
Epoch 2/3
23090/23090 [==============================]

In [ ]:
output_file = 'C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\Model\\accuracy_report_without_house_and_person.csv'

# Write results to CSV
with open(output_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Dropped Feature', 'Accuracy Loss'])
    for result in results:
        csvwriter.writerow(result)

In [ ]:
opener=open("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\Model\\accuracy_report_without_house_and_person.txt",'w')
for i in results:
    opener.write(str(i))
opener.close()